In [1]:
# --------------------------------------------------------
# Swin Transformer
# Copyright (c) 2021 Microsoft
# Licensed under The MIT License [see LICENSE for details]
# Written by Ze Liu
# --------------------------------------------------------

import os
import time
import argparse
import datetime
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist

from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.utils import accuracy, AverageMeter

from config import get_config
from models import build_model
from data import build_loader
from lr_scheduler import build_scheduler
from optimizer import build_optimizer
from logger import create_logger
from utils import load_checkpoint, save_checkpoint, get_grad_norm, auto_resume_helper, reduce_tensor
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import pandas as pd
import glob
from zipfile import ZipFile
import tqdm

In [ ]:
try:
    # noinspection PyUnresolvedReferences
    from apex import amp
except ImportError:
    amp = None
os.environ["CUDA_VISIBLE_DEVICES"] ="5"

def parse_option():
    parser = argparse.ArgumentParser('Swin Transformer training and evaluation script', add_help=False)
    parser.add_argument('--cfg', type=str, default="configs/swin_tiny_patch4_window7_224.yaml", metavar="FILE", help='path to config file', )
    parser.add_argument(
        "--opts",
        help="Modify config options by adding 'KEY VALUE' pairs. ",
        default=None,
        nargs='+',
    )

    # easy config modification
    parser.add_argument('--batch-size', type=int, help="batch size for single GPU")
    parser.add_argument('--data-path', type=str, help='path to dataset')
    parser.add_argument('--zip', action='store_true', help='use zipped dataset instead of folder dataset')
    parser.add_argument('--cache-mode', type=str, default='part', choices=['no', 'full', 'part'],
                        help='no: no cache, '
                             'full: cache all data, '
                             'part: sharding the dataset into nonoverlapping pieces and only cache one piece')
    parser.add_argument('--resume', default=True, help='resume from checkpoint')
    parser.add_argument('--accumulation-steps', type=int, help="gradient accumulation steps")
    parser.add_argument('--use-checkpoint', action='store_true',
                        help="whether to use gradient checkpointing to save memory")
    parser.add_argument('--amp-opt-level', type=str, default='O1', choices=['O0', 'O1', 'O2'],
                        help='mixed precision opt level, if O0, no amp is used')
    parser.add_argument('--output', default='deepfakedetection', type=str, metavar='PATH',
                        help='root of output folder, the full path is <output>/<model_name>/<tag> (default: output)')
    parser.add_argument('--tag', help='tag of experiment')
    parser.add_argument('--eval', default=True, help='Perform evaluation only')
    parser.add_argument('--base_lr', default=True, help='Perform evaluation only')
    parser.add_argument('--throughput', action='store_true', help='Test throughput only')

    # distributed training
    parser.add_argument("--local_rank", type=int, default=0, help='local rank for DistributedDataParallel')
#     parser.add_argument("-f", type=int,help='local rank for DistributedDataParallel')
    args, unparsed = parser.parse_known_args()

    config = get_config(args)

    return args, config


if __name__ == '__main__':
    
    _, config = parse_option()
    model = build_model(config)
    
    d = torch.load('ckpt_epoch_260.pth', map_location='cpu')
    model.load_state_dict(d['model'], strict=True)
    model = torch.nn.DataParallel(model)

    val_transform = A.Compose([ 
        A.Resize(height=512,width=512),
        A.CenterCrop(height=416, width=416),
        A.Normalize(

                    mean=[0.49313889,0.49313889, 0.49313889],
                    std=[0.36952964,0.36952964,0.36952964],
                    ),
                        ToTensorV2()
                    ])

    cudnn.benchmark = True
    dx = sorted(glob.glob('../data/test/Test/*.png')) # For private testing phase
    dx = sorted(glob.glob('../data/Val/*.png'))  # For validation

    model.eval()
    model.cuda()
    df = []
    for f in tqdm.tqdm(dx, total=len(dx)):
        im = cv2.imread(f)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        x = val_transform(image=im)['image'].unsqueeze(0).cuda()
        s = model(x)
        s = s.item()*100
        s = 0 if s<1 else s
        df.append([os.path.basename(f), s])
#             print(df[-1], int(s))

    df = pd.DataFrame(df)
    df.to_csv('predictions.csv', header=False, index=False)

    with ZipFile(f'sub_final.zip', 'w') as myzip:
        myzip.write('predictions.csv')   

=> merge config from configs/swin_tiny_patch4_window7_224.yaml


 76%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                   | 986/1301 [00:35<00:09, 31.96it/s]

'/home/jess/COV19DInfection/data/Val/Image_1300.png'